# FraudFinder - Exploratory Data Analysis

## Overview

In this notebook, you will perform exploratory data analysis on the historical bank transactions stored in BigQuery. Please make sure you have completed the [environment setup notebook](00_environment_setup.ipynb) prior to running this notebook.

Steps performed in this notebook:
- Read data from BigQuery
- Calculate summary statistics across historical transactions
- Calculate fraud percentages
- Plot distributions of transaction amounts
- Analyze customer-level aggregates of transaction data

## Setup

### Import libraries

In [ ]:
import sys

import pandas as pd
import plotly.express as px
import plotly.io as pio

In [ ]:
# Detect Cloud project from environment
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
pio.renderers.default = "iframe"

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

## Exploratory data analysis of transaction data in BigQuery

In this section, you'll explore some of the FraudFinder data by running queries and creating a couple interactive plots.

### Get transaction data summary statistics
First, you generate some summary statistics across the different fields in the transaction data.

In [ ]:
%%bigquery
SELECT
  COUNT(*) AS NUM_TX,
  MIN(TX_TS) AS MIN_TX_DATE,
  MAX(TX_TS) AS MAX_TX_DATE,
  COUNT(DISTINCT CUSTOMER_ID) AS NUM_CUSTOMERS,
  COUNT(DISTINCT TERMINAL_ID) AS NUM_TERMINALS,
  MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
  MAX(TX_AMOUNT) AS MAX_TX_AMOUNT
FROM
  tx.tx

The results show that there are over 148M transactions spanning the course of 2022 and 2025, taking place at 5000 different terminals with nearly 50K unique customers. The transaction amounts vary from 0 to over 1100, with the mean amount being about 55 dollars.

As of 2024, the data has been simulated into 2025, so that it is easier to do demonstrate some of the machine learning capabilities like evaluation in notebooks that you will be using later.

### Get fraud classification counts and percentages
Next, you look at the fraud label data to see how many (and what percentage of) transactions are classified as fraud or not.

In [ ]:
%%bigquery
SELECT
  TX_FRAUD,
  COUNT(*) AS NUM_TX,
  /* Calculates number of total transactions (not grouped) */
  SUM(COUNT(*)) OVER () AS OVR_TOTAL_TX,
  /* Calculates number of transaction in this group as % of total */
  SAFE_DIVIDE(
    COUNT(*),
    SUM(COUNT(*)) OVER ()
    ) AS PCT_TOTAL_TX
FROM
  tx.txlabels
GROUP BY
  TX_FRAUD
ORDER BY
  TX_FRAUD

You can see that about 2% of transactions are fraudulent, while the remaining 98% of transactions are not labeled as fraud.

### Plot transaction amount distribution
Next, you examine the distribution of transaction amounts more closely. After seeing above that the range is about 0 to 1100 dollars, you want to look at how often various transaction amounts across that range show up. Rather than bringing in all ~148M transactions amounts and plotting them, you can aggregate to the nearest dollar and count the number of observations in each dollar "bucket" using BigQuery.

In [ ]:
%%bigquery transaction_amount_dist
SELECT
  /* Round transaction to nearest dollar, use to group */
  ROUND(TX_AMOUNT, 0) AS ROUNDED_TX_AMOUNT,
  COUNT(*) AS NUM_TX
FROM
  tx.tx
GROUP BY
  ROUNDED_TX_AMOUNT
ORDER BY
  ROUNDED_TX_AMOUNT

In [ ]:
transaction_amount_dist

You can display the results as a histogram using the functionality to build interactive bar charts in [Plotly Express](https://plotly.com/python/bar-charts/).

In [ ]:
# Create histogram of transaction amounts
transaction_amount_interactive_histogram = px.bar(
    transaction_amount_dist,
    x="ROUNDED_TX_AMOUNT",
    y="NUM_TX",
    labels={
        "ROUNDED_TX_AMOUNT": "Transaction Amount (to nearest $)",
        "NUM_TX": "# of Transactions",
    },
    title="Number of Transactions by Amount",
    color_discrete_sequence=["blue"],
)

transaction_amount_interactive_histogram.update_traces(
    marker_color="black", marker_line_color="white", marker_line_width=0.1
)

transaction_amount_interactive_histogram.show()

You can see that the vast majority of transaction amounts are 125 dollars or less, with a steady "plateau" of 700K-800K transactions at each dollar amount from about 10 to 80. One advantage of the interactive bar chart is that you can zoom in to different areas (e.g. the bars are hard to see past 100 or so on the default plot, so reducing the y-axis helps) and then hover over different bars to see the actual numbers.

### Analyze customer-level aggregates of transaction data
After looking at the data on individual transactions in a couple different ways, you now look at the transaction data aggregated to the customer level. The query below gets three summary values for each customer in the dataset:
*   number of transactions
*   average transaction value
*   percentage of transactions that are fraudulent

In [ ]:
%%bigquery customer_aggregates
SELECT
  CUSTOMER_ID,
  COUNT(*) AS NUM_TX,
  AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,

  /* Get % of transactions that are fraudulent, accounting for
     possibility of missing fraud labels */
  SAFE_DIVIDE(
    SUM(IF(TX_FRAUD IS NOT NULL, TX_FRAUD, 0)),
    SUM(IF(TX_FRAUD IS NOT NULL, 1, 0))
    ) AS PCT_TX_FRAUD

FROM
  `tx.tx`
/* Join to labels data to get fraud status of each transaction */
LEFT JOIN
  `tx.txlabels` USING (TX_ID)
GROUP BY
  CUSTOMER_ID

In [ ]:
customer_aggregates.head()

You can now explore this customer data further using [Plotly's scatter plot functionality](https://plotly.com/python/line-and-scatter/) to look at each customer's average transaction value vs number of transactions in an interactive plot. The code below does that, while also coloring points by the percentage of a customer's transactions that are fraudulent.

In [ ]:
customer_aggregates_scatter_plot = px.scatter(
    customer_aggregates,
    x="NUM_TX",
    y="AVG_TX_AMOUNT",
    color="PCT_TX_FRAUD",
    labels={
        "CUSTOMER_ID": "Customer ID",
        "NUM_TX": "# of Transactions",
        "AVG_TX_AMOUNT": "Avg Transaction Amount ($)",
        "PCT_TX_FRAUD": "% of Transactions That Are Fraud",
    },
    hover_data=["CUSTOMER_ID"],
    title=(
        "<b>Customer-Level Transaction Aggregates</b><br>"
        + "(Each Point is 1 Customer, Colored by % of Fraud Transactions)"
    ),
)

customer_aggregates_scatter_plot.show()

The plot above shows that the vast majority of customers have about 6000 or fewer transactions with an average value of 100 or less, and most of those appear to have very low fraud rates. But there are customers at the far right with a very large number of transactions (in the 8000-11000 range), some of whom have very high average transaction values (from 100 to more than 300 dollars). And the customers in this group with very high transaction counts also have an extremely high rate of fraud: most upwards of 60%, with some higher than 90% fraud.

These trends found in the data are definitely worth keeping in mind as you think about how to construct a model to predict fraudulent transactions.

Note that because this is simulated data, the graph is unlikely to be very realistic (!) so consider this more of an exercise in understanding the workflow you may encounter and the products on Google Cloud that can help you solve for real-time fraud classification along the way.

## Next Step

Now you can go to the next notebook `02_feature_engineering_batch.ipynb`

Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.